In [ ]:
import pandas as pd
import requests

In [ ]:
json = requests.get("https://magisteriis.github.io/riot-games-help-to-swagger-dotnet/lcu/openapi.json").json()

In [ ]:
df = pd.DataFrame.from_dict(json['components']['schemas']['LolMatchHistoryMatchHistoryTeamBan']['properties'], orient='index')

In [ ]:
print(df['format'].fillna(df['type']).replace({
    'boolean': 'bool',
    'int64': 'i64',
    'int32': 'i32',
    'int16': 'i16',
    'uint64': 'u64',
    'uint32': 'u32',
    'uint16': 'u16',
    'string': 'String',
    'object': 'Value'
}).reset_index().apply(lambda x: x['index'] + ': ' + x['format'], axis=1).str.cat(sep=',\n'))

In [ ]:
translator = {
    'boolean': 'bool',
    'int64': 'i64',
    'int32': 'i32',
    'int16': 'i16',
    'uint64': 'u64',
    'uint32': 'u32',
    'uint16': 'u16',
    'string': 'String',
    'object': 'Value'
}

print((pd.DataFrame.from_dict(json['components']['schemas'], orient='index')
    ['properties']
    .dropna()
    .groupby(level=0)
    .apply(lambda x: pd.DataFrame.from_dict(x.squeeze(), orient='index'))
    .assign(
        ref=lambda x: x['$ref'].str.split('/').str[-1],
        format=lambda x: x['format']
            .fillna('Vec<' + 
                x
                ['items']
                .dropna()
                .apply(pd.Series)
                .assign(
                    ref=lambda y: y['$ref'].str.split('/').str[-1], 
                    format=lambda y: y['format'].fillna(y['type']).fillna(y['ref'])
                )
                ['format']
                .replace(translator) +
            '>')
            .fillna(x['type'])
            .fillna(x['ref']),
    )
    ['format']
    .replace(translator)
).groupby(level=0).apply(lambda x: f"""#[derive(Deserialize, Serialize, Debug)]
pub struct {x.index[0][0]} {{
    {',å    '.join(f'pub {i[1]}: {j}' if i[1] not in ['type', 'async'] else f'#[serde(alias = "{i[1]}")]å    pub {i[1]}_: {j}' for i, j in x.iteritems())}
}}""").str.cat(sep='\n\n').replace('å', '\n'))